### <ins>This notebook contains the following functions:</ins>

(function names can be clicked)

### [`local_mysql_Gans()`](#local_mysql_Gans) --> contains the credentials for accessing the local sql_Gans MySQL schema.
### [`city_data(cities)`](#city_data) --> collects data for the various cities of interest via webscraping wikipedia.
### [`population_update()`](#population_update) --> updates population data for the various cities of interest via webscraping wikipedia.
### [`city_weather()`](#city_weather) --> collects weather data for the cities present in the local sql_Gans schema via the openweather API.
### [`city_airports()`](#city_airports) --> collects data for the airports serving the various cities of interest.
### [`city_flights()`](#city_flights) --> collects data for the flights arriving to the airports stored in the local sql_Gans schema via the aerodatabox API.

### The MySQL schema for sql_Gans.
#### This code needs to be fed to the local MySQL instance.

```sql
-- Drop the database if it already exists
DROP DATABASE IF EXISTS sql_Gans ;

-- Create the database
CREATE DATABASE sql_Gans;

-- Use the database
USE sql_Gans;

-- Create the 'cities' table
CREATE TABLE cities (
    city_id INT AUTO_INCREMENT, -- Automatically generated ID for each city
    city VARCHAR(255) NOT NULL, -- Name of the city
    country VARCHAR(255) NOT NULL, -- Name of the city
    PRIMARY KEY (city_id) -- Primary key to uniquely identify each city
);

-- creates "coordinates" table

CREATE TABLE coordinates (
    city_id INT AUTO_INCREMENT, -- ID of the city
    latitude VARCHAR(255) NOT NULL, -- latitude of the city
    longitude VARCHAR(255) NOT NULL, -- longitude of the city
    FOREIGN KEY (city_id) REFERENCES cities(city_id) -- Foreign key to connect each population value to the respective city
);

-- create "population" table

CREATE TABLE population (
    id INT AUTO_INCREMENT, -- ID for each row
    city_id INT, -- ID of the city
    population INT UNSIGNED NOT NULL, -- Population of the city
    retrieval_date DATE NOT NULL, -- date the population value was retrieved
    PRIMARY KEY (id), -- Primary key to uniquely identify each row
    FOREIGN KEY (city_id) REFERENCES cities(city_id) -- Foreign key to connect each population value to the respective city
);

-- creates "weather_predictions" table

CREATE TABLE weather_predictions (
    id INT AUTO_INCREMENT, -- ID for each row
    city_id INT, -- ID of the city
    forecast_time_UTC DATETIME NOT NULL, -- future timepoint the weather prediction pertains to
    temperature FLOAT NOT NULL, -- temperature value
    apparent_temperature FLOAT NOT NULL, -- approximate feeling of temperature
    wind_speed FLOAT NOT NULL, -- wind speed
    rain_last_3h_mm FLOAT NOT NULL, -- amount of rain that fell in the previous 3 hours
    outlook VARCHAR(255) NOT NULL, -- how the weather conditions look like
    retrieval_time_UTC DATETIME NOT NULL, -- timepoint of prediction data retrieval
    PRIMARY KEY (id), -- Primary key to uniquely identify each row
    FOREIGN KEY (city_id) REFERENCES cities(city_id) -- Foreign key to connect each population value to the respective city
);

-- creates "airports" table

CREATE TABLE airports (
    airport_icao VARCHAR(5) NOT NULL, -- ICAO code for the airport
    airport_name VARCHAR(255) NOT NULL, -- name of the airport
    PRIMARY KEY (airport_icao), -- Primary key to uniquely identify each row
    UNIQUE (airport_icao) -- primary key columns can get only unique values
);

-- creates "cities_airports" table

CREATE TABLE cities_airports (
    id INT AUTO_INCREMENT, -- ID for each row
    airport_icao VARCHAR(5) NOT NULL, -- ICAO code for the airport
    city_id INT, -- ID of the city
    PRIMARY KEY (id), -- Primary key to uniquely identify each row
    FOREIGN KEY (city_id) REFERENCES cities(city_id), -- Foreign key to connect each airport to the respective city
    FOREIGN KEY (airport_icao) REFERENCES airports(airport_icao) -- Foreign key to connect each airport to its full name
);

-- creates the "flights" table

CREATE TABLE flights (
    id INT AUTO_INCREMENT, -- Automatically generated ID for each flight
    city_id INT, -- ID of the city
    arrival_icao VARCHAR(5) NOT NULL, -- ICAO code for the airport near the city of interest
    departure_icao VARCHAR(5), -- ICAO code for the airport where the flight is coming from
    departure_name VARCHAR(255), -- name of the airport
    flight_num VARCHAR(25) NOT NULL, -- identification number of the flight
    arrival_time_UTC DATETIME NOT NULL, -- timepoint of landing    
    retrieval_time_UTC DATETIME NOT NULL, -- timepoint of acquiring the data via the API
    PRIMARY KEY (id), -- Primary key to uniquely identify each row
    FOREIGN KEY (city_id) REFERENCES cities(city_id), -- Foreign key to connect each city-airport-flight combo to the respective city
    FOREIGN KEY (arrival_icao) REFERENCES airports(airport_icao)
);
```

### Installing dependencies.

In [127]:
# install if needed, depends on the local setup

#!pip install sqlalchemy
#!pip install pymysql

### Importing libraries and modules.

In [221]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import date, datetime, timedelta
from pytz import timezone
import json

### Credentials for connecting to the local MyQSL instance. <a id="local_mysql_Gans"></a>

In [1]:
# function to connect to the sql_Gans schema (present in a local MySQL instance)

def local_mysql_Gans():

    schema = "sql_Gans"
    host = "127.0.0.1"
    user = "root"
    password = "PASSWORD_HERE"
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    return connection_string

In [223]:
# storing the credentials for connecting to the local sql_Gans schema in a variable that can be called in subsequent functions 

connection_string_Gans = local_mysql_Gans()

### The cities of interest.

In [224]:
# the cities for which data will be scraped (a list is necessary to run the city_data() function)

city_list = ["Berlin", "Hamburg", "Munich", "Athens", "Manila", "London", "Paris", "Heidelberg"]

### Function for webscraping data from wikipedia on the cities of interest. <a id="city_data"></a>

In [225]:
# function for webscraping data from wikipedia on various cities

def city_data(cities):

    # all data come from wikipedia
    link_prefix = "https://en.wikipedia.org/wiki/"

    # an empty dictionary for storing the data retrieved from wikipedia 
    cities_data = {}

    # storing the date on which the data on the various cities were retrieved 
    retrieval_date = datetime.now().strftime("%Y-%m-%d")

    for city in cities:

        # creation of the url to be scraped.
        url = link_prefix + city
    
        # GET http request.
        response = requests.get(url)
    
        # applying some BeautifulSoup magic.
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # finding all html elements that store population values in the upper right table of the respective Wikipedia page and fetching the first element that corresponds to the city value.
        population = soup.find_all('td', class_ = 'infobox-data', string = re.compile("[0-9]{1,3},*[0-9]{1,3},[0-9]{1,3}"))[0].get_text()
    
        # finding all html elements that have the text "Country" in the upper right table of the respective Wikipedia page (there is only a single element) and fetching the next element that corresponds to the country. 
        country = soup.find_all('th', class_ = 'infobox-label', string = "Country")[0].find_next("td").get_text()
    
        # making webscraping function more robust by removing non-breaking spaces present in some cases e.g. Philippines
        country = country.replace(u"\xa0", u"")
        
        # finding the html element that stores the latitude of the city. 
        latitude = soup.find_all('span', class_ = 'latitude')[0].get_text()
    
        # finding the html element that stores the longitude of the city.
        longitude = soup.find_all('span', class_ = 'longitude')[0].get_text()
    
        # populates the cities_data dictionary with the requested details for each city
        cities_data[city] = [population, country, latitude, longitude, retrieval_date]

        # control point
        #print(f"{city} has {population} inhabitants and is located in {country} at {latitude} {longitude}.")

    # converts the cities_data dictionary to a data frame
    cities_df = (pd.DataFrame.from_dict(cities_data, orient = "index", columns=["Population", "Country", "Latitude", "Longitude", "retrieval_date"])
                   .reset_index()
                   .rename(columns = {"index" : "City"})
                )
    
    # removes commas from the population values
    cities_df["Population"] = pd.to_numeric(cities_df["Population"].str.replace(",", ""))
    
    # changes "retrieval_date" column to datetime
    cities_df["retrieval_date"] = pd.to_datetime(cities_df["retrieval_date"])

    # sending info to the "cities" table of the local MySQL sql_Gans schema
    # "cities" is updated only via this function and "city_id" column in the "cities" table is auto generated
    pd.DataFrame(cities_df[["City", "Country"]]).to_sql('cities',
                                                        if_exists='append',
                                                        con=connection_string_Gans,
                                                        index=False
                                                       )
    
    # sending info to the "coordinates" table of the local MySQL sql_Gans schema
    # "coordinates" is updated only via this function and "city_id" column in the "coordinates" is auto generated
    pd.DataFrame(cities_df[["Latitude", "Longitude"]]).to_sql('coordinates',
                                                              if_exists='append',
                                                              con=connection_string_Gans,
                                                              index=False
                                                             )

    # reading the "city_id" values present in the sql_Gans schema in order to pass them on to the "population" table
    # this must happen in a loop for the cities added in each function run, otherwise the "population" table will always get "city_id" values starting from zero
    cities_sql_df = pd.read_sql('''
                                select * from cities
                                ''',
                                con=connection_string_Gans
                               )

    # creation of an empty data frame that will store the "city_id values" for the cities added in each function run    
    cities_id_df = pd.DataFrame()
    
    # accessing the "city_id" value for each city present in the sql_Gans schema (for the cities added in each function run)
    for city in cities:
        city_id_df = pd.DataFrame(cities_sql_df.loc[cities_sql_df["city"] == city, "city_id"])    
        cities_id_df = pd.concat([cities_id_df, city_id_df], ignore_index = True)
    
    # inserting the "city_id" values from the "cities" table of the sql_Gans schema in the "cities_df" dataframe
    cities_df.insert(0, "city_id", cities_id_df["city_id"])
        
    # sending info to the "population" table of the local MySQL sql_Gans schema
    # "population" will be also updated via code placed outside of the present function (see population_update() function)
    pd.DataFrame(cities_df[["city_id", "Population", "retrieval_date"]]).to_sql('population',
                                                                                if_exists='append',
                                                                                con=connection_string_Gans,
                                                                                index=False
                                                                               )
    
    return "Tables \"cities\", \"coordinates\" and \"population\" have been updated."

### First run of the city_data() function.

In [226]:
city_data(city_list)

'Tables "cities", "coordinates" and "population" have been updated.'

### More cities of interest.

In [227]:
# more cities for which data will be scraped (a list is necessary to run the city_data() function)

city_list_2 = ["Frankfurt", "Madrid"]

### Second run of the city_data() function.

In [228]:
city_data(city_list_2)

'Tables "cities", "coordinates" and "population" have been updated.'

### Function for updating the population of the cities of interest. <a id="population_update"></a>

In [229]:
# function for updating the population values for cities already present in the local sql_Gans schema via webscraping from wikipedia

def population_update():

    # reading the cities present in the sql_Gans schema
    cities_from_sql_df = pd.read_sql('''
                                     select * from cities
                                     ''',
                                     con=connection_string_Gans
                                     )

    # storing the cities retrieved from sql_Gans into a list
    cities_list = cities_from_sql_df["city"].to_list()

    # storing the city IDs retrieved from sql_Gans into a list
    city_id_list = cities_from_sql_df["city_id"].to_list()

    # all data come from wikipedia
    link_prefix = "https://en.wikipedia.org/wiki/"

    # an empty dictionary for storing the data retrieved from wikipedia 
    cities_data = {}

    # storing the date on which the data on the various cities were retreived 
    retrieval_date = datetime.now().strftime("%Y-%m-%d")

    # zip() pulls together each pair of city IDs and city names in a tuple 
    for city in [x for x in zip(city_id_list, cities_list)]:

        # creation of the url to be scraped.
        url = link_prefix + city[1]
    
        # GET http request.
        response = requests.get(url)
    
        # applying some BeautifulSoup magic.
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # finding all html elements that store population values in the upper right table of the respective Wikipedia page and fetching the first element that corresponds to the city value.
        population = soup.find_all('td', class_ = 'infobox-data', string = re.compile("[0-9]{1,3},*[0-9]{1,3},[0-9]{1,3}"))[0].get_text()
    
        # populates the cities_data dictionary with the requested details for each city
        cities_data[city[1]] = [city[0], population, retrieval_date]

        # control point
        #print(f"{city[1]} has {population} inhabitants.")
        
    # converts the cities_data dictionary to a data frame
    cities_df = (pd.DataFrame.from_dict(cities_data, orient = "index", columns=["city_id", "Population", "retrieval_date"])
                   .reset_index()
                   .rename(columns = {"index" : "City"})
                )
    
    # removes commas from the population values
    cities_df["Population"] = pd.to_numeric(cities_df["Population"].str.replace(",", ""))
    
    # changes "retrieval_date" column to datetime
    cities_df["retrieval_date"] = pd.to_datetime(cities_df["retrieval_date"])

    # sending info to the "population" table of the local MySQL sql_Gans schema
    pd.DataFrame(cities_df[["city_id", "Population", "retrieval_date"]]).to_sql('population',
                                                                                if_exists='append',
                                                                                con=connection_string_Gans,
                                                                                index=False
                                                                               )
    
    return "Table \"population\" has been updated."

### Updating the population values in the local MySQL schema. 

In [230]:
population_update()

'Table "population" has been updated.'

### Function for collecting weather data for the cities of interest. <a id="city_weather"></a>

In [231]:
# function to collect weather data from the openweather API for the cities ALREADY present in the local sql_Gans schema and insert them back into the local sql_Gans schema

def city_weather():

    # reading the cities present in the sql_Gans schema
    cities_from_sql_df = pd.read_sql('''
                                     select * from cities
                                     ''',
                                     con=connection_string_Gans
                                     )

    # storing the cities retrieved from sql_Gans into a list
    cities_list = cities_from_sql_df["city"].to_list()

    # storing the city IDs retrieved from sql_Gans into a list
    city_id_list = cities_from_sql_df["city_id"].to_list()

    # creation of an empty data frame that will store the final result
    cities_weather_df = pd.DataFrame()

    # storing the time point (UTC) on which the data on the various cities were retrieved
    # openweather api reports weather forecast values for time point in UTC only
    utc_timezone = timezone('UTC')
    retrieval_time = datetime.now(utc_timezone).strftime("%Y-%m-%d %H:%M:%S")
    
    # key for accessing the openweather API
    API_key = "API_KEY_HERE"

    # zip() pulls together each pair of city IDs and city names in a tuple 
    for city in [x for x in zip(city_id_list, cities_list)]:
        
        # fetching weather data for a city from openweather
        forecast = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city[1]}&appid={API_key}&units=metric")

        # extracting info from the openweather response as a json
        forecast_json = forecast.json()

        # normalizing the "list" key of the forecast_json (contains the weather data) and creating a data frame that contains the weather data
        forecast_df = pd.json_normalize(forecast_json["list"])
        
        # extracting the weather evaluation from the "weather" column in order to create the "outlook" column
        forecast_df["outlook"] = [forecast_df["weather"][x][0]["main"] for x in range(len(forecast_df["weather"]))]

        # replacing NaN values in the rain propability column with 0 BUT only if the "rain.3h" column is already present
        # sometimes "rain.3h" values are not reported in the query, THEN the "rain.3h" column is added and populated with zeros
        if "rain.3h" in forecast_df.columns:
            forecast_df["rain.3h"] = forecast_df["rain.3h"].fillna(0)
        else:
            forecast_df["rain.3h"] = [0 for _ in range(len(forecast_df.index))]     
        
        # inserting the time point on which each weather prediction was retrieved
        forecast_df["retrieval_time_UTC"] = [retrieval_time for _ in range(len(forecast_df.index))]
        
        # cleaning up the forecast_df data frame by dropping columns with unnecessary data
        forecast_df.drop([name for name in forecast_df.columns if name not in ["dt_txt", "main.temp", "main.feels_like", "wind.speed", "rain.3h", "outlook", "retrieval_time_UTC"]], axis=1, inplace=True)

        # renaming the columns of the data frame
        # openweather api reports weather forecast values for timpe point in UTC only
        forecast_df.rename(columns = {"dt_txt" : "forecast_time_UTC",
                                      "main.temp" : "temperature",
                                      "main.feels_like" : "apparent_temperature",
                                      "wind.speed" : "wind_speed",
                                      "rain.3h" : "rain_last_3h_mm",
                                      "outlook" : "outlook",
                                      "retrieval_time_UTC" : "retrieval_time_UTC"
                                     }, inplace = True
                          )
        
        # inserting the "city_id" value from MySQL into the "forecast" data frame in a new column also named "city_id"
        forecast_df.insert(0, "city_id", [city[0] for _ in range(len(forecast_df.index))])
        
        # changing "forecast_time_UTC" column to datetime
        forecast_df["forecast_time_UTC"] = pd.to_datetime(forecast_df["forecast_time_UTC"])

        # changing "retrieval_time_UTC" column to datetime
        forecast_df["retrieval_time_UTC"] = pd.to_datetime(forecast_df["retrieval_time_UTC"])
        
        # inserting the weather values from forecast_df into the collective cities_weather_df data frame
        cities_weather_df = pd.concat([cities_weather_df, forecast_df], ignore_index = True)
        
    # inserting the collected weather values for all cities into the Gans MySQL schema
    cities_weather_df.to_sql('weather_predictions',
                             if_exists='append',
                             con=connection_string_Gans,
                             index=False
                            )
    
    return "Table \"weather_predictions\" has been updated."

### Collecting the weather data.

In [235]:
city_weather()

'Table "weather_predictions" has been updated.'

### Function for collecting airport data for the cities of interest. <a id="city_airports"></a>

In [233]:
# the function to collect data for the airports serving the cities ALREADY present in the local sql_Gans schema and insert them back into the local sql_Gans schema
# CAUTION: the tables "airports" and "cities_airports" of the sql_Gans schema must be dropped and recreated before each run of this function!!!!!!!!!!!!!

def city_airports():
 
    # reading the cities present in the sql_Gans schema
    cities_from_sql_df = (pd.read_sql('''
                                      select * from cities
                                      ''',
                                      con=connection_string_Gans
                                     )
                         )

    # storing the cities retrieved from MySQL into a list (order retained)
    cities_list = cities_from_sql_df["city"].to_list()

    # retrieving the coordinates for the cities in the Gans database
    city_coordinates_from_sql_df = (pd.read_sql('''
                                                select * from coordinates
                                                ''',
                                                con=connection_string_Gans
                                               )
                                   )
    # adding a column that will help allocate the correct negative or positive value to the latitude of each city
    city_coordinates_from_sql_df["latitude_description"] = city_coordinates_from_sql_df.apply(lambda row: "north" if "N" in row["latitude"] else "south",  axis = 1)

    # adding a column that will help allocate the correct negative or positive value to the longitude of each city
    city_coordinates_from_sql_df["longitude_description"] = city_coordinates_from_sql_df.apply(lambda row: "east" if "E" in row["longitude"] else "west",  axis = 1)

    # transforming the "latitude" column from string to numeric
    # CAUTION: the .str.split() below creates lists for each element of ["latitude"] that have empty elements in them
    # HOWEVER the transformation from string to numeric is successful and the required info is preserved
    city_coordinates_from_sql_df["latitude"] = pd.to_numeric(city_coordinates_from_sql_df.loc[:, "latitude"]
                                                                                         .str.split(r"\D", regex = True)
                                                                                         .apply(lambda x: x[0] + "." + x[1] + x[2])
                                                            )

    # transforming the "latitude" into a negative number if the respective city is located south of the equator
    city_coordinates_from_sql_df["latitude"] = city_coordinates_from_sql_df.apply(lambda row: -abs(row["latitude"]) if row["latitude_description"] == "south" else row["latitude"], axis = 1)

    # transforming the "longitude" column from string to numeric
    # CAUTION: the .str.split() below creates lists for each element of ["longitude"] that have empty elements in them
    # HOWEVER the transformation from string to numeric is successful and the required info is preserved
    city_coordinates_from_sql_df["longitude"] = pd.to_numeric(city_coordinates_from_sql_df.loc[:, "longitude"]
                                                                                          .str.split(r"\D", regex = True)
                                                                                          .apply(lambda x: x[0] + "." + x[1] + x[2])
                                                             )
    
    # transforming the "longitude" into a negative number if the respective city is located west of the Greenwich meridian
    city_coordinates_from_sql_df["longitude"] = city_coordinates_from_sql_df.apply(lambda row: -abs(row["longitude"]) if row["longitude_description"] == "west" else row["longitude"], axis = 1)

    # merging cities_from_sql_df and city_coordinates_from_sql_df
    cities_full_df = cities_from_sql_df.merge(city_coordinates_from_sql_df, how = "left", on = "city_id")

    # removing unnecessary columns from the "cities_full_df" data frame
    cities_full_df.drop(["latitude_description", "longitude_description"], axis=1, inplace=True)

    # assembling a list with city coordinates (latitude, longitude)
    cities_coordinates_list = [x for x in zip(cities_full_df["city_id"], cities_full_df["latitude"], cities_full_df["longitude"])]
    
    # creation of an empty data frame that will store the airports, which serve each city
    all_airports_df = pd.DataFrame()
    
    # iterating through each city's coordinates and collecting airport data for each one
    for element in cities_coordinates_list:
        
        # finding airports near the cities present in the Gans database through the aerodatabox API
        airport_url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        
        airport_querystring = {"lat" : element[1], "lon": element[2], "radiusKm" : "90", "limit" : "10", "withFlightInfoOnly" : "true"}

        airport_headers = {"X-RapidAPI-Key": "API_KEY_HERE",
	                       "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                          }

        airports = requests.get(airport_url, headers=airport_headers, params=airport_querystring)

        airports_json = airports.json()

        airports_df = pd.json_normalize(airports_json["items"])
        
        # renaming some columns of the "airports_df" data frame
        airports_df.rename(columns = {"icao" : "airport_icao",
                                      "name" : "airport_name",
                                     }, inplace = True
                          )
        
        # inserting the "city_id" value from MySQL into the "airports" data frame in a new column also named "city_id"
        airports_df.insert(0, "city_id", [element[0] for _ in range(len(airports_df.index))])
        
        # inserting the airport data from airports_df into the collective all_airports_df data frame
        all_airports_df = pd.concat([all_airports_df, airports_df], ignore_index = True)

    # filtering out duplicate entries since some airports might serve more than one city
    unique_airports_df = all_airports_df.loc[:, ["airport_icao", "airport_name"]].drop_duplicates(subset = ["airport_icao"], ignore_index=True)
    

    # inserting the unique airports into the local MySQL sql_Gans schema ("airports" table)
    pd.DataFrame(unique_airports_df[["airport_icao", "airport_name"]]).to_sql('airports',
                                                                              if_exists='append',
                                                                              con=connection_string_Gans,
                                                                              index=False
                                                                             )
    
    # inserting the collected airport data into the local MySQL sql_Gans schema ("cities airports" table)
    pd.DataFrame(all_airports_df[["city_id", "airport_icao"]]).to_sql('cities_airports',
                                                                      if_exists='append',
                                                                      con=connection_string_Gans,
                                                                      index=False
                                                                     )

    return "Tables \"airports\" and \"cities_airports\" have been updated."

### Collecting airport names and locations.

In [234]:
city_airports()

'Tables "airports" and "cities_airports" have been updated.'

### Function for collecting flight data from the airports near the cities of interest. <a id="city_flights"></a>

In [238]:
# the function to collect data for tomorrow's incoming flights to the airports serving the cities ALREADY present in the local sql_Gans schema and insert them back into the local sql_Gans schema
# the aerodatabox API returns time in UTC format or in the local timezone of the computer making the request

def city_flights():

    # reading the cities and associated airports present in the sql_Gans schema
    # note that some airports might serve more than one city
    cities_and_airports_from_sql_df = (pd.read_sql('''
                                                   select city_id, airport_icao from cities_airports
                                                   ''',
                                                   con=connection_string_Gans
                                                  )
                                      )

    # storing the cities retrieved from MySQL into a list
    cities_and_airports_list = list(zip(cities_and_airports_from_sql_df["city_id"], cities_and_airports_from_sql_df["airport_icao"]))

    # creation of an empty data frame that will store the final result
    all_arrivals_df = pd.DataFrame()

    # storing tomorrow's date in a string
    tomorrow = (date.today() + timedelta(1)).strftime("%Y-%m-%d")

    # storing the time point (UTC) on which the data on the various flights were retrieved
    # the UTC format of the aerodatabox API response is used in this function
    utc_timezone = timezone('UTC')
    retrieval_time = datetime.now(utc_timezone).strftime("%Y-%m-%d %H:%M:%S")

    # gathering data on the arriving flights to the airports near the cities of interest
    for element in cities_and_airports_list:

        # finding arriving flights near the cities present in the sql_Gans schema through the aerodatabox API
        # api url for the first half of the day (api query time ranges are limited to 12 hours)
        url1 = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{element[1]}/{tomorrow}T00:00/{tomorrow}T11:59"

        # api url for the second half of the day (api query time ranges are limited to 12 hours)
        url2 = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{element[1]}/{tomorrow}T12:00/{tomorrow}T23:59"

        # search is done only for arriving flights
        querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"true","withLocation":"false"}

        headers = {"X-RapidAPI-Key": "API_KEY_HERE",
    	           "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                  }

        # the html response for the first half of the day
        response1 = requests.get(url1, headers=headers, params=querystring)

        # the html response for the second half of the day
        response2 = requests.get(url2, headers=headers, params=querystring)

        # combining html responses for tomorrow in a list
        response_list = [response1, response2]

        # going through the full html response in search of flight data
        for response in response_list:

            # only successful get requests are considered (airports will report 204 if no flights are retrieved for the search criteria above)
            if response.status_code == 200:
        
                arrivals_df = pd.json_normalize(response.json()["arrivals"])
        
                # cleaning up the arrivals_df data frame by dropping columns with unnecessary data
                arrivals_df.drop([name for name in arrivals_df.columns if name not in ["number", "departure.airport.icao", "departure.airport.name", "arrival.scheduledTime.utc"]], axis=1, inplace=True)

                # renaming the columns of the data frame
                arrivals_df.rename(columns = {"number" : "flight_num",
                                              "departure.airport.icao" : "departure_icao",
                                              "departure.airport.name" : "departure_name",
                                              "arrival.scheduledTime.utc" : "arrival_time_UTC"
                                             }, inplace = True
                                  )

                # ordering the columns in data frame (useful for depicting the "arrivals_df" data frame in a jupyter notebook)
                arrivals_df = arrivals_df.reindex(columns=["departure_icao", "departure_name", "flight_num", "arrival_time_UTC"])

                # inserting the "airport_icao" value from MySQL into the "arrivals" data frame in a new column also named "arrival_icao"
                arrivals_df.insert(0, "arrival_icao", [element[1] for _ in range(len(arrivals_df.index))])

                # inserting the "city_id" value from MySQL into the "arrivals" data frame in a new column also named "city_id"
                arrivals_df.insert(0, "city_id", [element[0] for _ in range(len(arrivals_df.index))])

                # inserting the time point on which each arrival time was retrieved
                arrivals_df["retrieval_time_UTC"] = [retrieval_time for _ in range(len(arrivals_df.index))]
                        
                # changing "arrival_time_UTC" column to datetime
                arrivals_df["arrival_time_UTC"] = pd.to_datetime(arrivals_df["arrival_time_UTC"])

                # changing "retrieval_time_UTC" column to datetime
                arrivals_df["retrieval_time_UTC"] = pd.to_datetime(arrivals_df["retrieval_time_UTC"])
        
                # inserting the arrivals data from arrivals_df into the collective "all_arrivals_df" data frame
                all_arrivals_df = pd.concat([all_arrivals_df, arrivals_df], ignore_index = True)

            # if the get request for flight data failed, the next airport in the list is probed
            else:

                continue
                
    # inserting the collected airport data into the local sql_Gans schema ("cities airports" table)
    all_arrivals_df.to_sql('flights',
                           if_exists='append', 
                           con=connection_string_Gans, 
                           index=False)
    
    return "Table \"flights\" has been updated."

### Collecting data on inbound flights.

In [239]:
city_flights()

'Table "flights" has been updated.'